<a href="https://colab.research.google.com/github/LokeRuiKee/AChE-GNN/blob/main/DC_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install rdkit
!pip install colorama
!pip install deepchem

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import time
import copy
import pickle

import numpy as np
import pandas as pd
from IPython.core.display import display, HTML

from sklearn.metrics import mean_squared_error, r2_score, make_scorer

import deepchem as dc

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [ ]:
def rmse(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return np.sqrt(mean_squared_error(y_eval.flatten(), y_hat.flatten()))


def r2(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return r2_score(y_eval.flatten(), y_hat.flatten())


def peason_r(estimator, X_eval, y_eval):
    y_hat = estimator.predict(X_eval)
    return np.corrcoef(y_eval.flatten(), y_hat.flatten())[0, 1]


def peason_r_metric(y_true, y_pred):
    return np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]

peason_r_score = make_scorer(peason_r_metric)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create train/test datasets

In [ ]:
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Load the dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/tx2c00283_si_002/SupplementalFiles/Human_dataset_continuous.xlsx"
df = pd.read_excel(file_path)

# Specify the columns
smiles_column = "SMILES"
y_column = "continuous-value"

# Split the data into features and target
X = df[smiles_column]
y = df[y_column]

# Split the data into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Concatenate the features and target for each set
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Define the directory and file paths
base_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/data/process/continuous"
train_file = os.path.join(base_dir, "pdY_train.csv")
val_file = os.path.join(base_dir, "pdY_val.csv")
test_file = os.path.join(base_dir, "pdY_test.csv")

# Create the directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Save the data to CSV files
train_data.to_csv(train_file, index=False)
val_data.to_csv(val_file, index=False)
test_data.to_csv(test_file, index=False)

In [ ]:
train_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/data/process/continuous/pdY_train.csv"
test_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/data/process/continuous/pdY_test.csv"

tasks = ["continuous-value"]
ntasks = len(tasks)
featurizer_func = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader(tasks=tasks, feature_field='SMILES', featurizer=featurizer_func)

train_dataset = loader.create_dataset(train_file)
test_dataset = loader.create_dataset(test_file)

# Default

In [ ]:
model = dc.models.GraphConvModel(ntasks, mode='regression', model_dir="models/graph_conv_01")
model.fit(train_dataset, nb_epoch=50)

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

#pickle.dump(model, open("models/graph_conv_01.pkl", "wb"))

Train RMSE: 0.44364540879546344
Train Pearson's R: 0.9581131093057201
Test RMSE: 0.8857301805576576
Test Pearson's R: 0.7992666551549412


In [ ]:
model = dc.models.GraphConvModel(ntasks, mode='regression', model_dir="models/graph_conv_01")
model.restore()

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

Train RMSE: 0.44364540879546344
Train Pearson's R: 0.9581131093057201
Test RMSE: 0.8857301805576576
Test Pearson's R: 0.7992666551549412


## `graph_conv_layers = [64, 64, 64]`, `dense_layer_size=128`, `dropout=0`

In [ ]:
# to fix TypeError: 'Metric' object is not subscriptable in ori code

import deepchem as dc

# Define the metric
metric = dc.metrics.Metric(dc.metrics.rms_score)

# Create a list of metrics if needed
metrics = [metric]

# Define the callback with the correct metric handling
callback = dc.models.ValidationCallback(test_dataset, 100, metrics=metrics)

# Fit the model
model.fit(train_dataset, nb_epoch=100, callbacks=[callback])

# Calculate and print the RMSE
train_rmse = metric.compute_metric(model.predict(train_dataset), train_dataset.y)
test_rmse = metric.compute_metric(model.predict(test_dataset), test_dataset.y)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

Step 200 validation: rms_score=3.25054
Step 300 validation: rms_score=2.31172
Step 400 validation: rms_score=1.61092
Step 500 validation: rms_score=1.26434
Step 600 validation: rms_score=1.15431
Step 700 validation: rms_score=1.10221
Step 800 validation: rms_score=1.05446
Step 900 validation: rms_score=1.02411
Step 1000 validation: rms_score=0.99771
Step 1100 validation: rms_score=0.989117
Step 1200 validation: rms_score=0.971996
Step 1300 validation: rms_score=0.950726
Step 1400 validation: rms_score=0.956249
Step 1500 validation: rms_score=0.941443
Step 1600 validation: rms_score=0.937108
Step 1700 validation: rms_score=0.940545
Step 1800 validation: rms_score=0.939824
Step 1900 validation: rms_score=0.941739
Step 2000 validation: rms_score=0.943693
Step 2100 validation: rms_score=0.931249
Step 2200 validation: rms_score=0.913999
Step 2300 validation: rms_score=0.926782
Step 2400 validation: rms_score=0.907563
Step 2500 validation: rms_score=0.909721
Step 2600 validation: rms_score=0

In [ ]:
print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

Train RMSE: 0.37144376933490914
Train Pearson's R: 0.9697497545140984
Test RMSE: 0.9010889858024458
Test Pearson's R: 0.7983182892311323


Train RMSE: 0.37144376933490914
Train Pearson's R: 0.9697497545140984
Test RMSE: 0.9010889858024458
Test Pearson's R: 0.7983182892311323

significant high of RMSE in test compared to train shows overfitting.

In [ ]:
# inference
from rdkit import Chem

# example test case from validation set
# correct value = 7.185087

# Define a new SMILES string
new_smiles = "Cc1cc(=O)oc2cc(OCc3cc[n+](Cc4ccccc4F)cc3)ccc12"

# Create a ConvMolFeaturizer
featurizer = dc.feat.ConvMolFeaturizer()

# Convert the SMILES string to the format expected by the model
new_mol = Chem.MolFromSmiles(new_smiles)
new_data = featurizer.featurize([new_mol])

# Create a Dataset object from the new data
new_dataset = dc.data.NumpyDataset(new_data)

# Predict the class probabilities for the new data
new_pred_probs = model.predict(new_dataset)

# Select the probabilities of the first task
new_pred_probs = new_pred_probs[:, 0]

print("Predicted probs for new data:", new_pred_probs[0])

Predicted probs for new data: 6.725976


Note to self: what does the continuous value means

In [ ]:
#ori code (output TypeError: 'Metric' object is not subscriptable)

# batch_size = 64
# batches_per_epoch = 600/batch_size
# learning_rate = dc.models.optimizers.ExponentialDecay(0.0001, 0.9, batches_per_epoch)

# model = dc.models.GraphConvModel(ntasks,
#                                  graph_conv_layers=[64, 64, 64],
#                                  dense_layer_size=128,
#                                  dropout=0.,
#                                  batch_normalize=True,
#                                  mode='regression',
#                                  batch_size=batch_size,
#                                  learning_rate=learning_rate)

# metric = dc.metrics.Metric(dc.metrics.rms_score)
# callback = dc.models.ValidationCallback(test_dataset, 100, metric)
# model.fit(train_dataset, nb_epoch=100, callbacks=callback)

# print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
# print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

# print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
# print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

In [ ]:
# model = dc.models.GraphConvModel(ntasks, mode='regression')
# model.restore()

# print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
# print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

# print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
# print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

##  `graph_conv_layers = [128, 128, 128,  128, 128]`, `dense_layer_size=256`, `dropout=0.0`

In [ ]:

# Define the model
model = dc.models.GraphConvModel(
    n_tasks=ntasks,
    graph_conv_layers=[128, 128, 128, 128, 128],
    dense_layer_size=256,
    dropout=0.0,
    batch_normalize=True,
    mode='regression',
    batch_size=64
)

# Define the metric
metric = dc.metrics.Metric(dc.metrics.rms_score)
metrics = [metric]

# Create the callback
callback = dc.models.ValidationCallback(test_dataset, 100, metrics=metrics)

# Fit the model
model.fit(train_dataset, nb_epoch=100, callbacks=[callback])

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

Step 100 validation: rms_score=1.44599
Step 200 validation: rms_score=1.41144
Step 300 validation: rms_score=1.23664
Step 400 validation: rms_score=1.04291
Step 500 validation: rms_score=0.940565
Step 600 validation: rms_score=0.906373
Step 700 validation: rms_score=0.846281
Step 800 validation: rms_score=0.860708
Step 900 validation: rms_score=0.856451
Step 1000 validation: rms_score=0.806116
Step 1100 validation: rms_score=0.81079
Step 1200 validation: rms_score=0.826208
Step 1300 validation: rms_score=0.824628
Step 1400 validation: rms_score=0.815268
Step 1500 validation: rms_score=0.823608
Step 1600 validation: rms_score=0.844555
Step 1700 validation: rms_score=0.822599
Step 1800 validation: rms_score=0.82032
Step 1900 validation: rms_score=0.834175
Step 2000 validation: rms_score=0.80763
Step 2100 validation: rms_score=0.819438
Step 2200 validation: rms_score=0.807328
Step 2300 validation: rms_score=0.810937
Step 2400 validation: rms_score=0.783598
Step 2500 validation: rms_score=

In [ ]:
# ori
model = dc.models.GraphConvModel(ntasks,
                                 graph_conv_layers=[128, 128, 128, 128, 128],
                                 dense_layer_size=256,
                                 dropout=0.0,
                                 batch_normalize=True,
                                 mode='regression',
                                 batch_size=64)

metric = dc.metrics.Metric(dc.metrics.rms_score)
callback = dc.models.ValidationCallback(test_dataset, 100, metric)
model.fit(train_dataset, nb_epoch=100, callbacks=callback)

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

In [ ]:
# # ori
# y_test_pred = model.predict(test_dataset)
# test_pred_df = pd.DataFrame({"smiles": test_dataset.ids, "actual": test_dataset.y.flatten(), "pred": y_test_pred.flatten()})
# test_pred_df.to_csv("/content/drive/MyDrive/Colab Notebooks/Dataset/data/process/continuous/results/test_pred.csv", index=False)

## graph_conv_layers = [64, 128,], dense_layer_size=128, dropout=0.0

In [ ]:
model = dc.models.GraphConvModel(ntasks,
                                 graph_conv_layers=[512, 512, 512],
                                 dense_layer_size=512,
                                 dropout=0.,
                                 batch_normalize=True,
                                 mode='regression',
                                 batch_size=64)

# Define the metric
metric = dc.metrics.Metric(dc.metrics.rms_score)
metrics = [metric]

# Create the callback
callback = dc.models.ValidationCallback(test_dataset, 100, metrics=metrics)

# Fit the model
model.fit(train_dataset, nb_epoch=100, callbacks=[callback])

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

Step 100 validation: rms_score=3.47723
Step 200 validation: rms_score=4.7227
Step 300 validation: rms_score=3.30143
Step 400 validation: rms_score=1.78038
Step 500 validation: rms_score=1.44158
Step 600 validation: rms_score=0.987652
Step 700 validation: rms_score=0.925598
Step 800 validation: rms_score=0.868425
Step 900 validation: rms_score=0.846627
Step 1000 validation: rms_score=0.810435
Step 1100 validation: rms_score=0.833678
Step 1200 validation: rms_score=0.886114
Step 1300 validation: rms_score=0.837756
Step 1400 validation: rms_score=0.851968
Step 1500 validation: rms_score=0.803544
Step 1600 validation: rms_score=0.801711
Step 1700 validation: rms_score=0.80513
Step 1800 validation: rms_score=0.796152
Step 1900 validation: rms_score=0.829404
Step 2000 validation: rms_score=0.78321
Step 2100 validation: rms_score=0.810472
Step 2200 validation: rms_score=0.794862
Step 2300 validation: rms_score=0.837624
Step 2400 validation: rms_score=0.803296
Step 2500 validation: rms_score=0

In [ ]:
# ori
model = dc.models.GraphConvModel(ntasks,
                                 graph_conv_layers=[512, 512, 512],
                                 dense_layer_size=512,
                                 dropout=0.,
                                 batch_normalize=True,
                                 mode='regression',
                                 batch_size=64)

metric = dc.metrics.Metric(dc.metrics.rms_score)
callback = dc.models.ValidationCallback(test_dataset, 100, metric)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

print("Train RMSE:", rmse(model, train_dataset, train_dataset.y))
print("Train Pearson's R:", peason_r(model, train_dataset, train_dataset.y))

print("Test RMSE:", rmse(model, test_dataset, test_dataset.y))
print("Test Pearson's R:", peason_r(model, test_dataset, test_dataset.y))

In [ ]:
# inference with model 3
from rdkit import Chem

# example test case from validation set
# correct value = 7.185087

# Define a new SMILES string
new_smiles = "Cc1cc(=O)oc2cc(OCc3cc[n+](Cc4ccccc4F)cc3)ccc12"

# Create a ConvMolFeaturizer
featurizer = dc.feat.ConvMolFeaturizer()

# Convert the SMILES string to the format expected by the model
new_mol = Chem.MolFromSmiles(new_smiles)
new_data = featurizer.featurize([new_mol])

# Create a Dataset object from the new data
new_dataset = dc.data.NumpyDataset(new_data)

# Predict the class probabilities for the new data
new_pred_probs = model.predict(new_dataset)

# Select the probabilities of the first task
new_pred_probs = new_pred_probs[:, 0]

print("Predicted probs for new data:", new_pred_probs[0])

Predicted probs for new data: 7.150532


In [ ]:
# to write the comparison of actual and predicted value into csv file.

from datetime import datetime
import pytz

y_test_pred = model.predict(test_dataset)
test_pred_df = pd.DataFrame({"smiles": test_dataset.ids, "actual": test_dataset.y.flatten(), "pred": y_test_pred.flatten()})

# Define Malaysia timezone
malaysia_tz = pytz.timezone('Asia/Kuala_Lumpur')

# Get the current date and time in Malaysia timezone
current_time_malaysia = datetime.now(malaysia_tz).strftime("%Y%m%d_%H%M%S")

# Create the filename with the date and time appended
filename = f"test_pred_{current_time_malaysia}.csv"

# Save the DataFrame to the specified path with the new filename
test_pred_df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/Dataset/data/process/continuous/results/{filename}", index=False)